In [1]:
import pandas as pd

In [2]:
predictions = pd.read_csv(f"../data/predictions/predictions-{11}-{19}.csv")
final_predictions = pd.read_csv(f"../data/predictions/predictions-final-{11}-{17}.csv")

In [3]:
features = pd.read_csv("../data/tsfeatures/tsfeatures.csv")\
    .dropna(axis = 1, how = 'all')

features = features.rename({"unique_id":"store_item"}, axis=1)

In [4]:
final_feats = pd.read_csv(f"../data/tsfeatures/tsfeatures_final.csv")\
    .dropna(axis = 1, how = 'all')
    
final_feats = final_feats.rename({"unique_id":"store_item"}, axis=1)

In [5]:
from sklearn.impute import KNNImputer

def impute_hurst(df):
    
    imputer = KNNImputer()
    knn_impute = df.select_dtypes(include=['float64'])
    knn_imputed = imputer.fit_transform(knn_impute)
    knn_imputed = pd.DataFrame(knn_imputed, columns=knn_impute.columns)
    return knn_imputed[["hurst", "entropy"]]

features[["hurst", "entropy"]] = impute_hurst(features)
final_feats[["hurst", "entropy"]] = impute_hurst(final_feats)

In [6]:
def drop_same_val_colums(df):
    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    return df.drop(cols_to_drop, axis=1)

features = drop_same_val_colums(features)
final_feats = drop_same_val_colums(final_feats)

In [7]:
predictions = predictions.rename({'Unnamed: 0':"date"}, axis=1)
predictions.date = pd.to_datetime(predictions.date)

final_predictions = final_predictions.rename({'Unnamed: 0':"date"}, axis=1)
final_predictions.date = pd.to_datetime(final_predictions.date)

In [8]:
from helper import create_features

In [9]:
error = predictions.iloc[:, 3:9] # fcast_models
predictions.drop(predictions.iloc[:, 3:9], inplace=True, axis=1)
errors = error.sub(predictions.sales, axis=0)
model_least_err = pd.DataFrame(errors.abs().idxmin(axis=1), columns=["model_w_least_error"])
predictions = pd.concat([predictions, create_features(predictions), model_least_err], axis=1)

In [10]:
final_predictions = pd.concat([final_predictions, create_features(final_predictions)], axis=1)

In [11]:
final_df = pd.merge(predictions, features, right_on="store_item", left_on= "store_item")
y = final_df.model_w_least_error
X = final_df.drop(final_df.columns[[0, 1, 2, 19]], inplace=False, axis=1)

In [12]:
y.to_csv("../data/forecasts_for_ml/y.csv", index=False)
X.to_csv("../data/forecasts_for_ml/X.csv", index=False)

In [13]:
final_predictions = pd.concat([final_predictions, create_features(final_predictions)], axis=1)

In [14]:
final_df = pd.merge(final_predictions, final_feats, right_on="store_item", left_on= "store_item")

final_df = final_df[X.columns]
final_df.to_csv("../data/forecasts_for_ml/X_final.csv", index=False)